# Depurado de Datos

En esta sección se detalla el proceso de depuración inicial aplicado a los datos de análisis. Se emplea un procedimiento **ETL** (Extract, Transform, Load), lo que implica que se creará una nueva fuente de datos. A partir de los archivos JSON originales, se generará un nuevo conjunto de datos (Data Set) que luego será cargado y procesado en otro programa.

Los datos recogidos de forma continua, como la frecuencia cardíaca, la frecuencia respiratoria, el estrés o la batería corporal, pueden presentar valores duplicados en la misma marca temporal o valores faltantes en la serie. El código en esta sección ofrece una solución para abordar estos posibles problemas.

Además, ciertos valores pueden resultar poco significativos debido a fallos en los sensores, interfiriendo en la claridad del análisis. Un ejemplo son los valores de frecuencia cardíaca anormalmente altos o bajos y los datos de estrés captados durante el entrenamiento.

La depuración abarca el tratamiento de estos problemas mediante cuatro pasos clave:

- **Eliminación de datos duplicados:** Se identificarán y eliminarán valores duplicados en una misma marca temporal.

- **Relleno de datos faltantes:** Los valores faltantes en la serie temporal se detectarán y rellenarán siguiendo un criterio, usando el valor anterior conocido.

- **Descarte de datos anómalos de frecuencia cardíaca:** Los valores de frecuencia cardíaca superiores a 220 bpm o inferiores a 40 bpm (rango estimado para deportistas) se considerarán anómalos y se eliminarán.

- **Eliminación de datos de estrés captados durante el entrenamiento:** El periodo de actividad en días de entrenamiento se definirá manualmente, y durante ese tiempo se eliminarán posibles datos de estrés.


In [2]:
import os
import json
import pandas as pd


#Carpetas de entrada y salida
carpeta_brutos = "C:/Users/marco/Downloads/datosBrutosTFG/usuario1"
carpeta_procesados = "C:/Users/marco/Downloads/datosProcesadosTFG/usuario1"

#Diccionario donde se especifican manualmente los entrenamientos para los archivo de estrés
entrenamientos = {
    "reloj_36_health_stress_20241007.json": ("19:45", "01:36"),  
    "reloj_36_health_stress_20241008.json": ("19:03", "01:36"),  
    "reloj_36_health_stress_20241010.json": ("19:52", "01:28"),  
    "reloj_36_health_stress_20241011.json": ("18:07", "01:31"),  
    "reloj_36_health_stress_20241014.json": ("17:37", "01:26"),  
    "reloj_36_health_stress_20241015.json": ("17:33", "01:15"),  
    "reloj_36_health_stress_20241017.json": ("17:57", "01:15"),  
    "reloj_36_health_stress_20241018.json": ("17:35", "01:21"),  
}

def convertir_a_segundos(hora_str):
    #Pasamos el formato de hora:minuto a segundos
    horas, minutos = map(int, hora_str.split(":"))
    return horas * 3600 + minutos * 60

def eliminar_duplicados(data):
    #Función para eliminar duplicados, dependiendo de si la estructura del JSON es lista o diccionario.
    if isinstance(data, list):
        resultado = []
        tiempos_previos = set()  #Usamos la estructura set para evitar duplicados
        for muestra in data:
            if muestra["startTimeInSeconds"] not in tiempos_previos:
                resultado.append(muestra)
                tiempos_previos.add(muestra["startTimeInSeconds"])
        return resultado
    elif isinstance(data, dict):
        resultado = {}
        indices_previos = set()
        for indice, valor in data.items():
            if indice not in indices_previos:
                resultado[indice] = valor
                indices_previos.add(indice)
        return resultado


def rellenar_faltantes(data, step=1):
    #Función que rellena los valores faltantes con el valor anterior conocido
    indices = sorted(map(int, data.keys()))
    resultado = {}

    for i in range(indices[0], indices[-1] + step, step):
        if str(i) in data:
            resultado[str(i)] = data[str(i)]
        else:
            resultado[str(i)] = resultado[str(i - step)]

    return resultado


def procesar_entrenamiento(data):
    #Archivos de entrenamiento. Elimina duplicados y descarta valores FC fuera del rango (40, 220).
    for item in data:
        if "samples" in item:
            item["samples"] = eliminar_duplicados(item["samples"])
            
            item["samples"] = [
                muestra for muestra in item["samples"]
                if 40 <= muestra.get("heartRate", 0) <= 220 
            ]
    return data


def procesar_resumen_salud(data):
    #Archivos de Resumen de Salud. Elimina duplicados y rellena faltantes.
    for item in data:
        for summary in item.get("summaries", []):
            if "epochSummaries" in summary:
                summary["epochSummaries"] = eliminar_duplicados(summary["epochSummaries"])
                summary["epochSummaries"] = rellenar_faltantes(summary["epochSummaries"], step=1)
    return data

def procesar_estres(data):
    #Archivos de Estrés y Body Battery. Elimina duplicados y rellena faltantes.
    end_time = start_time + duration

    for item in data:
        item["timeOffsetStressLevelValues"] = eliminar_duplicados(item["timeOffsetStressLevelValues"])
        item["timeOffsetStressLevelValues"] = rellenar_faltantes(item["timeOffsetStressLevelValues"], step=180)

        item["timeOffsetBodyBatteryValues"] = eliminar_duplicados(item["timeOffsetBodyBatteryValues"])
        item["timeOffsetBodyBatteryValues"] = rellenar_faltantes(item["timeOffsetBodyBatteryValues"], step=180)

    return data

def procesar_estres_entrenamiento(data, start_time, duration):
    #Archivos de Estrés y Body Battery. Elimina duplicados y rellena faltantes.
    end_time = start_time + duration

    for item in data:
        item["timeOffsetStressLevelValues"] = eliminar_duplicados(item["timeOffsetStressLevelValues"])
        item["timeOffsetStressLevelValues"] = rellenar_faltantes(item["timeOffsetStressLevelValues"], step=180)

        item["timeOffsetBodyBatteryValues"] = eliminar_duplicados(item["timeOffsetBodyBatteryValues"])
        item["timeOffsetBodyBatteryValues"] = rellenar_faltantes(item["timeOffsetBodyBatteryValues"], step=180)

        #Si hay valores de estrés durante el entrenamiento, los cambiamos por -1
        for timestamp, valor in item["timeOffsetStressLevelValues"].items():
            timestamp = int(timestamp)
            if start_time <= timestamp <= end_time and valor > 0:
                item["timeOffsetStressLevelValues"][str(timestamp)] = -1
    return data

def procesar_estres(data):
    #Archivos de Estrés y Body Battery. Elimina duplicados y rellena faltantes.
    end_time = start_time + duration

    for item in data:
        item["timeOffsetStressLevelValues"] = eliminar_duplicados(item["timeOffsetStressLevelValues"])
        item["timeOffsetStressLevelValues"] = rellenar_faltantes(item["timeOffsetStressLevelValues"], step=180)

        item["timeOffsetBodyBatteryValues"] = eliminar_duplicados(item["timeOffsetBodyBatteryValues"])
        item["timeOffsetBodyBatteryValues"] = rellenar_faltantes(item["timeOffsetBodyBatteryValues"], step=180)

    return data


#Bucle principal. Llamada a funciones y gestión de directorios (lectura y guardado)
for filename in os.listdir(carpeta_brutos):
    #Leemos los datos brutos
    if filename.endswith(".json"):
        with open(os.path.join(carpeta_brutos, filename), "r", encoding="utf-8") as file:
            data = json.load(file)
        
        if "health_stress" in filename and filename in entrenamientos:
            #Convertimos el tiempo de inicio y duración de formato "HH:MM" a segundos
            start_time_str, duration_str = entrenamientos[filename]
            start_time = convertir_a_segundos(start_time_str)
            duration = convertir_a_segundos(duration_str)
            
            data = procesar_estres_entrenamiento(data, start_time, duration)
        elif "health_stress" in filename and filename not in entrenamientos:
            data = procesar_estres(data)
        elif "activity_detail" in filename:
            data = procesar_entrenamiento(data)
        elif "health_snapshot" in filename:
            data = procesar_resumen_salud(data)

        #Guardamos los datos ya procesados
        procesados = os.path.join(carpeta_procesados, filename)
        with open(procesados, "w", encoding="utf-8") as outfile:
            json.dump(data, outfile, ensure_ascii=False, indent=4)

print("Archivos procesados y guardados en la carpeta de salida.")


Archivos procesados y guardados en la carpeta de salida.
